Lambda School Data Science

*Unit 2, Sprint 3, Module 4*

---


# Model Interpretation 2

You will use your portfolio project dataset for all assignments this sprint.

## Assignment

Complete these tasks for your project, and document your work.

- [ ] Continue to iterate on your project: data cleaning, exploratory visualization, feature engineering, modeling.
- [ ] Make a Shapley force plot to explain at least 1 individual prediction.
- [ ] Share at least 1 visualization (of any type) on Slack.

But, if you aren't ready to make a Shapley force plot with your own dataset today, that's okay. You can practice this objective with another dataset instead (any dataset you've worked with previously). An example solution will be provided for the Titanic dataset, which is in the data directory of this repository.

**Multi-class classification** will result in multiple sets of Shapley Values (one for each class).


## Stretch Goals
- [ ] Make Shapley force plots to explain at least 4 individual predictions.
    - If your project is Binary Classification, you can do a True Positive, True Negative, False Positive, False Negative.
    - If your project is Regression, you can do a high prediction with low error, a low prediction with low error, a high prediction with high error, and a low prediction with high error.
- [ ] Use Shapley values to display verbal explanations of individual predictions.
- [ ] Use the SHAP library for other visualization types.

The [SHAP repo](https://github.com/slundberg/shap) has examples for many visualization types, including:

- Force Plot, individual predictions
- Force Plot, multiple predictions
- Dependence Plot
- Summary Plot
- Summary Plot, Bar
- Interaction Values
- Decision Plots

We just did the first type during the lesson. The [Kaggle microcourse](https://www.kaggle.com/dansbecker/advanced-uses-of-shap-values) shows two more. Experiment and see what you can learn!


## Links
- [Kaggle / Dan Becker: Machine Learning Explainability — SHAP Values](https://www.kaggle.com/learn/machine-learning-explainability)
- [Christoph Molnar: Interpretable Machine Learning — Shapley Values](https://christophm.github.io/interpretable-ml-book/shapley.html)
- [SHAP repo](https://github.com/slundberg/shap) & [docs](https://shap.readthedocs.io/en/latest/)

In [38]:
%%capture
import sys

# If you're on Colab:
if 'google.colab' in sys.modules:
    DATA_PATH = 'https://raw.githubusercontent.com/LambdaSchool/DS-Unit-2-Applied-Modeling/master/data/'
    !pip install category_encoders==2.*
    !pip install eli5
    !pip install pdpbox
    !pip install shap

# If you're working locally:
else:
    DATA_PATH = '../data/'

In [39]:
# all imports needed for this sheet

import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import category_encoders as ce
from sklearn.ensemble import RandomForestClassifier
from sklearn.impute import SimpleImputer
from sklearn.pipeline import make_pipeline
from sklearn.feature_selection import f_regression, SelectKBest
from sklearn.linear_model import Ridge
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
from sklearn.model_selection import validation_curve
from sklearn.tree import DecisionTreeRegressor
import xgboost as xgb


import seaborn as sns
from sklearn.metrics import accuracy_score

from sklearn.model_selection import GridSearchCV, RandomizedSearchCV


In [40]:
pip install category_encoders

Note: you may need to restart the kernel to use updated packages.


In [41]:
df = pd.read_excel(DATA_PATH+'/Unit_2_project_data.xlsx')

In [42]:
exit_reasons = ['Rental by client with RRH or equivalent subsidy', 
                'Rental by client, no ongoing housing subsidy', 
                'Staying or living with family, permanent tenure', 
                'Rental by client, other ongoing housing subsidy',
                'Permanent housing (other than RRH) for formerly homeless persons', 
                'Staying or living with friends, permanent tenure', 
                'Owned by client, with ongoing housing subsidy', 
                'Rental by client, VASH housing Subsidy'
               ]

In [43]:
 # create target column (multiple types of exits to perm)
df['perm_leaver'] = df['3.12 Exit Destination'].isin(exit_reasons)

In [44]:
# replace spaces with underscore
df.columns = df.columns.str.replace(' ', '_')

In [45]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

train = df

# Split train into train & test
train, test = train_test_split(train, train_size=0.80, test_size=0.20, 
                              stratify=train['perm_leaver'], random_state=42)

def wrangle(X):
    """Wrangle train, validate, and test sets in the same way"""
    
    # Prevent SettingWithCopyWarning
    X = X.copy()
    
    # drop any private information
    X = X.drop(columns=['3.1_FirstName', '3.1_LastName', '3.2_SocSecNo', 
                      '3.3_Birthdate', 'V5_Prior_Address'])
    
    # drop unusable columns
    X = X.drop(columns=['2.1_Organization_Name', '2.4_ProjectType',
                        'WorkSource_Referral_Most_Recent', 'YAHP_Referral_Most_Recent',
                        'SOAR_Enrollment_Determination_(Most_Recent)',
                        'R7_General_Health_Status', 'R8_Dental_Health_Status',
                        'R9_Mental_Health_Status', 'RRH_Date_Of_Move-In',
                        'RRH_In_Permanent_Housing', 'R10_Pregnancy_Due_Date',
                        'R10_Pregnancy_Status', 'R1_Referral_Source',
                        'R2_Date_Status_Determined', 'R2_Enroll_Status',
                        'R2_Reason_Why_No_Services_Funded', 'R2_Runaway_Youth',
                        'R3_Sexual_Orientation', '2.5_Utilization_Tracking_Method_(Invalid)',
                        '2.2_Project_Name', '2.6_Federal_Grant_Programs', '3.16_Client_Location',
                        '3.917_Stayed_Less_Than_90_Days', 
                        '3.917b_Stayed_in_Streets,_ES_or_SH_Night_Before', 
                        '3.917b_Stayed_Less_Than_7_Nights', '4.24_In_School_(Retired_Data_Element)',
                        'CaseChildren', 'ClientID', 'HEN-HP_Referral_Most_Recent',
                        'HEN-RRH_Referral_Most_Recent', 'Emergency_Shelter_|_Most_Recent_Enrollment',
                        'ProgramType', 'Days_Enrolled_Until_RRH_Date_of_Move-in',
                        'CurrentDate', 'Current_Age', 'Count_of_Bed_Nights_-_Entire_Episode',
                        'Bed_Nights_During_Report_Period'])
        
    # drop rows with no exit destination (current guests at time of report)
    X = X.dropna(subset=['3.12_Exit_Destination'])
    
    # remove columns to avoid data leakage
    X = X.drop(columns=['3.12_Exit_Destination', '5.9_Household_ID', '5.8_Personal_ID',
                       '4.2_Income_Total_at_Exit', '4.3_Non-Cash_Benefit_Count_at_Exit'])
    
    # Drop needless feature
    unusable_variance = ['Enrollment_Created_By', '4.24_Current_Status_(Retired_Data_Element)']
    X = X.drop(columns=unusable_variance)

    # Drop columns with timestamp
    timestamp_columns = ['3.10_Enroll_Date', '3.11_Exit_Date', 
                         'Date_of_Last_ES_Stay_(Beta)', 'Date_of_First_ES_Stay_(Beta)', 
                         'Prevention_|_Most_Recent_Enrollment', 'PSH_|_Most_Recent_Enrollment', 
                         'Transitional_Housing_|_Most_Recent_Enrollment', 'Coordinated_Entry_|_Most_Recent_Enrollment', 
                         'Street_Outreach_|_Most_Recent_Enrollment', 'RRH_|_Most_Recent_Enrollment', 
                         'SOAR_Eligibility_Determination_(Most_Recent)', 'Date_of_First_Contact_(Beta)',
                         'Date_of_Last_Contact_(Beta)', '4.13_Engagement_Date', '4.11_Domestic_Violence_-_When_it_Occurred',
                         '3.917_Homeless_Start_Date']
    X = X.drop(columns=timestamp_columns)
    
    # drop rows with no bednights
    X = X.dropna(subset=['Days_Enrolled_in_Project', 
           'Count_of_Bed_Nights_(Housing_Check-ins)', 
          'Age_at_Enrollment', 'CaseMembers',
          '4.12_Contact_Services'])
    
    # return the wrangled dataframe
    return X



In [46]:
train = wrangle(train)
test = wrangle(test)

In [47]:
# Assign to X, y
features = ['Days_Enrolled_in_Project', 
           'Age_at_Enrollment', 'CaseMembers'
          ]
target = 'perm_leaver'
X_train = train[features]
y_train = train[target]
X_test = test[features]
y_test = test[target]


In [48]:
X_train.shape

(1505, 3)

In [49]:
y_train.shape

(1505,)

In [50]:
X_test.shape

(365, 3)

In [51]:
y_test.shape

(365,)

In [54]:
from scipy.stats import randint, uniform
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import RandomizedSearchCV

param_distributions = { 
    'n_estimators': randint(5, 500, 5), 
    'max_depth': [10, 15, 20, 50, None], 
    'max_features': uniform(0, 1), 
}

search = RandomizedSearchCV(
    RandomForestRegressor(random_state=42), 
    param_distributions=param_distributions, 
    n_iter=20, 
    cv=5, 
    scoring='neg_mean_absolute_error', 
    verbose=10, 
    return_train_score=True, 
    n_jobs=-1, 
    random_state=42
)

search.fit(X_train, y_train);

Fitting 5 folds for each of 20 candidates, totalling 100 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:   19.8s
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:   20.6s
[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:   22.7s
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:   24.3s
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:   27.2s
[Parallel(n_jobs=-1)]: Done  45 tasks      | elapsed:   28.5s
[Parallel(n_jobs=-1)]: Done  56 tasks      | elapsed:   30.8s
[Parallel(n_jobs=-1)]: Done  69 tasks      | elapsed:   32.6s
[Parallel(n_jobs=-1)]: Done  82 tasks      | elapsed:   35.3s
[Parallel(n_jobs=-1)]: Done  96 out of 100 | elapsed:   38.2s remaining:    1.6s
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:   39.2s finished


In [55]:
print('Best hyperparameters', search.best_params_)
print('Cross-validation MAE', -search.best_score_)
model = search.best_estimator_

Best hyperparameters {'max_depth': 50, 'max_features': 0.9385527090157502, 'n_estimators': 395}
Cross-validation MAE 0.30689052365132097


In [ ]:
y_train.mean()

In [ ]:
# Get an individual observation to explain.
# For example, the 0th row from the test set.
row = X_test.iloc[[0]]
row

In [ ]:
# Did this guest exit to perm housing?
y_test.iloc[[0]]

In [ ]:
# What does the model predict for this guest?
model.predict(row)

In [ ]:
# Why did the model predict this?
import shap

explainer = shap.TreeExplainer(model)
shap_values = explainer.shap_values(row)

shap.initjs()
fig = shap.force_plot(
    base_value=explainer.expected_value,
    shap_values=shap_values,
    features=row,
    show=False

)

plt.savefig('shap.png')

In [ ]:
import shap
import matplotlib.pyplot as plt

shap.initjs()

explainer = shap.TreeExplainer(model)
shap_values = explainer.shap_values(row)
fig = shap.summary_plot(shap_values, train, show=False)
plt.savefig('shap.png')

In [ ]:
shap_values

In [ ]:
def predict(Days_Enrolled_in_Project,  
            Age_at_Enrollment, CaseMembers, model):

  # Make df from inputs
  df = pd.DataFrame(
      data=[[Days_Enrolled_in_Project, 
            Age_at_Enrollment, CaseMembers,            
             ]],
      columns=['Days_Enrolled_in_Project', 
              'Age_at_Enrollment', 'CaseMembers']      
  )

  # Make a prediction
  pred = model.predict(df)[0]
  
  # Calculate the shap values
  explainer = shap.TreeExplainer(model)
  shap_values = explainer.shap_values(df)

  # Print some results
  feature_names = df.columns
  feature_values = df.values[0]
  shaps = pd.Series(shap_values[0], zip(feature_names, feature_values))

  result = f'${pred:,.0f} estimate exit destination for this guest. \n\n'
  result += f'Starting from baseline of {explainer.expected_value:,.0f} \n'
  result += shaps.to_string()
  print(result)

  # Show the shapley force plot
  shap.initjs()
  return shap.force_plot(
    base_value=explainer.expected_value,
    shap_values=shap_values,
    features=df
)
  
predict(30, 20, 3, model)

In [ ]:
predict(10, 10, 2, model)

In [ ]:
row.values